In [7]:
from objects import BallOnPlate

import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import numpy
import multiprocessing

import os
import sys

import pickle
import time

import neat

from time import sleep
import math
import numpy as np

simulation_seconds = 20.

def eval_genome(genome):
    ballOnPlate = BallOnPlate(showGUI=False, randomInitial=False)

    global_cost = 1e10

    CONST_VALUE = 0.7
    intial_positions = [
        [CONST_VALUE, CONST_VALUE],
        [-CONST_VALUE, -CONST_VALUE],
        [-CONST_VALUE, CONST_VALUE],
        [CONST_VALUE, -CONST_VALUE],
#         [0., 0.]
    ]

    reference_positions = [
        [-CONST_VALUE, -CONST_VALUE],
        [CONST_VALUE, CONST_VALUE],
        [CONST_VALUE, -CONST_VALUE],
        [-CONST_VALUE, CONST_VALUE],
#         [0., 0.]
    ]

    for i in range(len(intial_positions)):

        reference_point = np.array(reference_positions[i])
        initial_position = np.array(intial_positions[i])
        
#         while abs(length_rate) < 0.4:
#             reference_point = np.random.randint(-9, 9, 2) / 10.
#             initial_position = np.random.randint(-9, 9, 2) / 10.


#     #         dx = reference_positions[i][0] - intial_positions[i][0]
#     #         dy = reference_positions[i][1] - intial_positions[i][1]
        length_rate = np.linalg.norm(
            np.array(reference_point) - np.array(initial_position)
        )
        
        result = 0
        dropDown = False

        ref_point  = reference_point

        posOnPlate = ballOnPlate.reset(initial_position)
        prevPosOnPlate = posOnPlate

        prev_err    = np.array([0, 0])
        integr_err  = [0, 0]

        ### PID controller
        prop    = float(genome[0])
        diff    = float(genome[1]) / BallOnPlate.D_T
        integr  = float(genome[2]) * BallOnPlate.D_T

        while ballOnPlate.time < simulation_seconds:
            # Get error
            err = ref_point - posOnPlate
            result -= np.sqrt(err[0] * err[0] + err[1] * err[1]) * (ballOnPlate.time + 1) / length_rate

            integr_err += err
            d_err = err - prev_err

            control = prop * err + diff * d_err + integr_err * integr
            control = np.array([-control[1], control[0]])
            
#             envInput[0] = prop * err[1] + diff * d_err[1] + integr_err[1] * integr
#             envInput[0] = -envInput[0]
#             envInput[1] = prop * err[0] + diff * d_err[0] + integr_err[0] * integr

            prev_err = err
            ### PID controller

            prevPosOnPlate = posOnPlate

            posOnPlate, isEnd = ballOnPlate.step(control)
            if isEnd:
                # Bad penalty as fall
                dropDown = True
                break
            # sleep(ballOnPlate.dt)
        
        DROP_PENALTY_VALUE = 1e3

        if dropDown:
            current_cost = ballOnPlate.time * (DROP_PENALTY_VALUE/(2*simulation_seconds)) + result - DROP_PENALTY_VALUE
        else:
            current_cost = ballOnPlate.time * (DROP_PENALTY_VALUE/(2*simulation_seconds)) + result

#         global_cost += current_cost
        global_cost = min(current_cost, global_cost)

    ballOnPlate.close()
#     global_cost /= float(len(intial_positions))
    return global_cost / 100, 

def checkBoundsZero():
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in range(len(child)):
                    if child[i] < 0:
                        child[i] = 0
            return offspring
        return wrapper
    return decorator


In [8]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
# toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("attr_float", random.random)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_float, 3)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", eval_genome)

# register the crossover operator
toolbox.register("mate", tools.cxUniform, indpb=0.7)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutGaussian, indpb=0.1, mu=0, sigma=.02)

toolbox.decorate("mate", checkBoundsZero())
toolbox.decorate("mutate", checkBoundsZero())

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=7)

In [10]:
random.seed(time.time())

# Process Pool of 4 workers
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
toolbox.register("map", pool.map)

pop = toolbox.population(n=50)
hof = tools.HallOfFame(5)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

try:
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=1000, 
                        stats=stats, halloffame=hof)
except KeyboardInterrupt:
    print('Try-catch')

pool.close()

print("-- End of (successful) evolution --")

best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

# Best individual is [0.8314504111768981, 1.1925901500923737, 0.016085473303319286]

# 

gen	nevals	avg    	std    	min     	max      
0  	50    	-10.595	1.94397	-18.8688	-0.488594
1  	31    	-9.21082	3.77618	-18.5544	-0.488594
2  	29    	-4.12426	4.60227	-10.4406	-0.488594
3  	32    	-0.48613	0.0484076	-0.68101	-0.336334
4  	34    	-0.460356	0.231638 	-1.99183	-0.275294
5  	32    	-0.368045	0.209193 	-1.42028	0.0230882
6  	36    	-0.256565	0.111768 	-0.436754	0.0230882
7  	28    	-0.0964623	0.188784 	-0.804963	0.246271 
8  	33    	0.0771682 	0.0839786	-0.0605224	0.246271 
9  	32    	0.19779   	0.0788839	0.0230882 	0.365729 
10 	34    	0.277932  	0.0514425	0.133632  	0.365729 
11 	32    	0.327183  	0.133915 	-0.583293 	0.376282 
12 	33    	0.353037  	0.0821276	-0.136213 	0.461644 
13 	35    	0.390107  	0.0546334	0.101744  	0.475748 
14 	32    	0.443925  	0.0946199	-0.158189 	0.564272 
15 	34    	0.516751  	0.0428637	0.439189  	0.601151 
16 	27    	0.569414  	0.0398411	0.461644  	0.730328 
17 	28    	0.567318  	0.279483 	-1.16077  	0.775466 
18 	29    	0.713282  	0.0523488	